In [ ]:
# recalc_opts.py
def recalc_opts(ib, df):
    '''Recalculates option prices and margins
    Arg:
       (ib) as connection object
       (df) as the dataframe of options
    Returns: dataframe with updated OptPrices and OptMargins
    '''
    opt_contracts = [Contract(conId=c) for c in df.optId]
    qopts = ib.qualifyContracts(*opt_contracts)
    tickers = ib.reqTickers(*qopts)
    optPrices = {t.contract.conId: t.marketPrice() for t in tickers} # {symbol: undPrice}
    optMargins = {c.conId: ib.whatIfOrder(c,Order(action='SELL', orderType='MKT', totalQuantity=abs(ls), whatIf=True)).initMarginChange 
        for c, ls in (zip(qopts, df.lotsize))}
    
    # update prices and margins
    df = df.set_index('optId')
    df.optPrice.update(pd.Series(optPrices))
    df.optMargin.update(pd.Series(optMargins))
    df.optMargin = df.optMargin.astype('float')
    df = df.assign(rom=df.optPrice*df.lotsize/df.optMargin*252/df.dte)
    
    return df.reset_index()

In [ ]:
import pandas as pd

from os import listdir
from ib_insync import *
util.startLoop()

from helper import get_nse_remqty, get_connected, grp_opts, get_prec, recalc_opts

fspath = '../data/nse/'  # path for nse pickles

minOptPrice = 0.15
minRom = 0.25
nLargest = 5

fs = listdir(fspath)

# get the remaining quantities
with get_connected('nse', 'live') as ib:
    remqty = get_nse_remqty(ib)

optsList = [f for f in fs if f[-3:] == 'pkl']

cols = ['optId', 'symbol', 'right', 'expiration', 'dte', 'strike', 'undPrice', 
'lo52', 'hi52', 'Fall', 'Rise', 'loFall', 'hiRise', 'std3', 'loStd3', 'hiStd3', 
'lotsize', 'optPrice', 'optMargin', 'rom']

df1 = pd.concat([pd.read_pickle(fspath+f) 
                 for f in optsList], axis=0, sort=True).reset_index(drop=True)[cols]

# filter for high probability
df2 = df1[((df1.strike > df1.hi52) | 
           (df1.strike < df1.lo52)) & 
          (df1.optPrice > minOptPrice) & 
          (df1.rom > minRom)]

df2 = df2.sort_values('rom', ascending=False)
df2[df2.rom > minRom].reset_index(drop=True)

In [ ]:
# recalculated the prices and margins
with get_connected('nse', 'live') as ib:
    df2 = recalc_opts(ib, df2)

In [ ]:
# group by the largest N return on margin
df2 = grp_opts(df2)
df3 = df2.groupby('symbol').apply(lambda x: x.nlargest(nLargest, 'rom'))

df4 = df3.assign(expPrice=[get_prec(p*1.1, 0.05) for p in df3.optPrice])
df4 = df4.assign(remqty=[remqty[u] for u in df4.symbol])
df4 = df4.assign(expQty=df4.remqty)

In [ ]:
df4 = df4[df4.remqty != 0] # remove zero remaining quantities (blacklisted)

In [ ]:
df4

In [ ]:
# make calls and puts watch - to quickly weed out risky options
gb = df5.groupby('right')

if 'C' in [k for k in gb.indices]:
#     df5_calls = gb.get_group('C').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, True])
    df5_calls = gb.get_group('C').reset_index(drop=True)
    
    df5_callsymbols = df5_calls.symbol.unique()
    watchcalls = [('DES', s, 'STK', 'NSE') if s not in ['NIFTY50', 'BANKNIFTY'] else ('DES', s, 'IND', 'NSE') for s in df5_callsymbols]
    df5_wp = util.df(watchcalls)
    df5_wp.to_csv(fspath+'callswatch.csv', index=None, header=False)
    
else:
    df5_calls = pd.DataFrame([]) # empty dataframe

if 'P' in [k for k in gb.indices]:
#     df5_puts = gb.get_group('P').reset_index(drop=True).sort_values(['symbol', 'strike'], ascending=[True, False])
    df5_puts = gb.get_group('P').reset_index(drop=True)
    
    # make watchlist
    df5_putssymbols = df5_puts.symbol.unique()
    watchputs = [('DES', s, 'STK', 'NSE') if s not in ['NIFTY50', 'BANKNIFTY'] else ('DES', s, 'IND', 'NSE') for s in df5_putssymbols]
    df5_wp = util.df(watchputs)
    df5_wp.to_csv(fspath+'putswatch.csv', index=None, header=False)
else:
    df5_puts = pd.DataFrame([]) # empty dataframe

# output the consolidated puts and calls dataframe
df5 = pd.concat([df5_puts, df5_calls]).reset_index(drop=True)
df5.to_csv(fspath+'check.csv', index=None, header=True)

In [ ]:
df5 = df5.assign(expQty=3)

In [ ]:
df5.loc[df5.symbol == 'BSOFT', 'expPrice'] = 0.55

In [ ]:
df5

In [ ]:
sum(df5.expQty*df5.expPrice*df5.lotsize)

In [ ]:
opt_contracts = [Contract(conId=c) for c in df5.optId]
with get_connected('nse', 'live') as ib:
    qopts = ib.qualifyContracts(*opt_contracts)

In [ ]:
orders = [LimitOrder('SELL', qty*lotsize, price) for qty, lotsize, price in zip(df5.expQty, df5.lotsize, df5.expPrice)]

In [ ]:
with get_connected('nse', 'live') as ib:
    limitTrade = [ib.placeOrder(c, o) for c, o in zip(qopts,  orders)]

In [ ]:
# ib = get_connected('nse', 'live')

In [ ]:
[ib.placeOrder(c, o) for c, o in zip(qopts,  orders)]

In [ ]:
ib.reqGlobalCancel()

In [ ]:
import pandas as pd
df_temp = pd.read_pickle('../data/nse/BHARATFIN.pkl')

In [ ]:
df_temp

In [ ]:
df_temp.loc[:, 'lotsize'] = 500

df_temp.to_pickle('../data/nse/BHARATFIN.pkl')